## RNN 모델 N-Byte 방식 (함수정보 포함 vs 미포함 => 1:1 비율)

## (1) 데이터로드

In [1]:
# (1) 데이터로드
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기
bin8_3 = pd.read_csv("../../바이너리_최종데이터_1004/gcc8/o3/o3_bincore8.csv", index_col=0)
print(bin8_3.shape)

# reset_index (hex processing 하면서 값이 빠졌으니까 + n_gram 에서 index를 다루기 때문에)
bin8_3.reset_index(inplace=True, drop=True)

print('shape -', bin8_3.shape)
print('reset_index 완료')
print('input data shape')
bin8_3.head()

(31784379, 2)
shape - (31784379, 2)
reset_index 완료
input data shape


,bin,label
0,47,0
1,108,0
2,105,0
3,98,0
4,47,0


In [2]:
# (2-1) 데이터체크 1 - hex(16진수)가 256 label을 가져야 dummies 변환 가능 
# 16진수 256개 종류가 있어서 pd.get_dummies 사용 가능.
print(len(bin8_3['bin'].unique()))

# (2-2) 데이터 체크 2 - 1, 0 비율 ==> 1이 함수의 갯수를 뜻함
# 정답 데이터 1, 0 비율 확인  ==> 1이 함수의 갯수를 뜻함
print(bin8_3['label'].value_counts())

256
0    31747485
1       36894
Name: label, dtype: int64


## (3) N Byte씩 자르기

In [3]:
idx_bin = bin8_3[bin8_3['label']==1].index  # 407, 474 ...
ls_bin = list(idx_bin)

# 최종 뽑을 행에 대한 index
ls_idx_bin = []

# n byte 자르기 방식
left_idx, right_idx = 0, 96 # 3개씩

# n byte 자르기
for k in range(left_idx, right_idx):
    ls_idx_bin.extend(list(idx_bin + k)) # index 형이라서 가능

#ls_idx = list(set(ls_idx)) 
ls_idx_bin.sort() # 인덱스 정렬

# 1차 index 해당범위 초과한 것들 없애기
ls_idx_bin = list(filter(lambda x: x<len(bin8_3), ls_idx_bin))
print(len(ls_idx_bin))

# 2차 남은 index들 중 right_idx 나눈 나머지 없애기
sub_bin = len(ls_idx_bin)%(right_idx)
print('나머지', sub_bin)

ls_idx_bin = ls_idx_bin[:len(ls_idx_bin)-sub_bin]
print('최종 길이', len(ls_idx_bin))

print('bin8_3', len(ls_idx_bin))

# loc 로 수정필요
bin8_3_Ngram = bin8_3.loc[ls_idx_bin,:].copy()

3541824
나머지 0
최종 길이 3541824
bin8_3 3541824


## (4) false data 만들기

In [4]:
# false data 만들기 - False 데이터 랜덤 생성

# 목표치
goal_bin = len(bin8_3_Ngram)/right_idx
count_bin = 0

print(goal_bin)

# 최종 데이터 Frame
d_bin = pd.DataFrame(columns = bin8_3.columns)

binutils_df = []
# goal 에 도달할 때까지
while True:
    if (count_bin == goal_bin):
            break
    # 진행상황 살펴보기 위함
            
    # 랜덤 N 바이트씩 뽑음
    # random index
    random_idx_bin = np.random.randint(len(bin8_3)-right_idx)

    if count_bin % 1000==0:
        print(count_bin, end=' ')
        print(random_idx_bin)

    df_bin = bin8_3[random_idx_bin : random_idx_bin + right_idx]
    
    # 뽑은 index의 N 바이트 중에 1이 없는 경우만
    if 1 not in df_bin['label'] and count_bin < goal_bin:
        binutils_df.append(df_bin)
        count_bin+=1

print('완료')
print(len(binutils_df))

36894.0
0 23273160
1000 6105510
2000 4119578
3000 7588683
4000 29251327
5000 19391460
6000 31332509
7000 15180557
8000 3547855
9000 23301708
10000 6535875
11000 15154552
12000 28660947
13000 9090921
14000 18660397
15000 2122594
16000 22450299
17000 29011796
18000 7339087
19000 5741126
20000 13846478
21000 26546066
22000 6524959
23000 457390
24000 29471475
25000 9013042
26000 13333491
27000 2319890
28000 22318221
29000 7705772
30000 19345600
31000 13874550
32000 13116455
33000 24309979
34000 21153919
35000 3261603
36000 7312436
완료
36894


In [5]:
# True data와 False Data 같은지 체크
print(len(binutils_df))
print(bin8_3['label'].value_counts()[1])

36894
36894


## (5) False Data + True Data 합치기

In [6]:
f_data = pd.concat(binutils_df)
final = pd.concat([f_data, bin8_3_Ngram])
final.shape

(7083648, 2)

## (6) one hot encoding

In [7]:
# 훈련데이터 (gcc 최적화버전 0, 1, 2, 3 one hot encoding)
bc8_3_onehot_Ngram = pd.get_dummies(final['bin'])
bc8_3_onehot_Ngram = pd.concat([final['label'], bc8_3_onehot_Ngram], axis=1)

print('원핫인코딩완료')
print(bc8_3_onehot_Ngram.shape)

원핫인코딩완료
(7083648, 257)


In [9]:
# 훈련 데이터, 훈련 라벨
x_bc8_3 = bc8_3_onehot_Ngram.iloc[:,1:].to_numpy()
y_bc8_3 = bc8_3_onehot_Ngram['label'].to_numpy()
print(x_bc8_3.shape, x_bc8_3.shape)

x_bc8_3 = x_bc8_3.reshape(-1, right_idx, x_bc8_3.shape[1])
y_bc8_3 = y_bc8_3.reshape(-1, right_idx, 1)

print(x_bc8_3.shape, y_bc8_3.shape)

(7083648, 256) (7083648, 256)
(73788, 96, 256) (73788, 96, 1)


In [10]:
# numpy 행, 열 섞기
p = np.random.permutation(x_bc8_3.shape[0])

x_bc8_3 = x_bc8_3[p]
y_bc8_3 = y_bc8_3[p]

print(x_bc8_3.shape, y_bc8_3.shape)

(73788, 96, 256) (73788, 96, 1)


In [11]:
np.save('x_bc8_3', x_bc8_3)
np.save('y_bc8_3', y_bc8_3)

## (7) 모델

In [14]:
# (10) 양방향 LSTM 모델링 작업
from tensorflow.keras import layers, models
#from tf.keras.models import Model, Sequential
#from tf.keras.layers import SimpleRNN, Input, Dense, LSTM
#from tf.keras.layers import Bidirectional, TimeDistributed

# 학습
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 10) # 조기종료 콜백함수 정의

xInput = layers.Input(batch_shape=(None,right_idx, 256)) 
xBiLstm = layers.Bidirectional(layers.LSTM(144, return_sequences=True, stateful=False), merge_mode = 'concat')(xInput)
xOutput = layers.TimeDistributed(layers.Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

## (8) 학습

In [15]:
model1 = models.Model(xInput, xOutput)
model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model1.summary()

import time
print('======Training stage======')
starttime = time.time()

model1.fit(x_bc8_3,
           y_bc8_3,
           epochs = 500,
           batch_size = 32,
           validation_split=0.3,
           callbacks=[early_stopping])

endtime = time.time()

print('걸린시간:', endtime - starttime)

model1.save('gcc8_bin_core_s96_h144_o3.h5')
print('save 완료')

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 96, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 96, 288)           461952    
_________________________________________________________________
time_distributed (TimeDistri (None, 96, 1)             289       
Total params: 462,241
Trainable params: 462,241
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 51651 samples, validate on 22137 samples
Epoch 1/500
51651/51651 [==============================] - 46s 893us/sample - loss: 0.0107 - accuracy: 0.9972 - val_loss: 0.0031 - val_accuracy: 0.9991
Epoch 2/500
51651/51651 [==============================] - 32s 611us/sample - loss: 0.0022 - accuracy: 0.9994 - val_loss: 0.0019 - val_accuracy: 0.9995
Epoc

걸린시간: 750.4405198097229
save 완료
